In [3]:
%matplotlib widget

In [2]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import LassoSelector
from matplotlib.widgets import RectangleSelector
from matplotlib.path import Path

class image_lasso_selector:
    def __init__(self, img, mask_alpha=.75, figsize=(10,10)):
        """
        img must have shape (X, Y, 3)
        """
        self.img = img
        self.mask_alpha = mask_alpha
        plt.ioff() # see https://github.com/matplotlib/matplotlib/issues/17013
        self.fig = plt.figure(figsize=figsize)
        self.ax = self.fig.gca()
        self.displayed = self.ax.imshow(img)
        plt.ion()
        
        self.lasso = LassoSelector(self.ax, self.onselect, useblit=False)
        self.lasso.set_visible(True)
        
        pix_x = np.arange(self.img.shape[0])
        pix_y = np.arange(self.img.shape[1])
        xv, yv = np.meshgrid(pix_y,pix_x)
        self.pix = np.vstack( (xv.flatten(), yv.flatten()) ).T
        
        self.mask = np.zeros(self.img.shape[:2])
        
    def onselect(self, verts):
        self.verts = verts
        p = Path(verts)
        self.indices = p.contains_points(self.pix, radius=0).reshape(self.mask.shape)
        self.draw_with_mask()
        
    def draw_with_mask(self):
        array = self.displayed.get_array().data

        # https://en.wikipedia.org/wiki/Alpha_compositing#Straight_versus_premultiplied     
        self.mask[self.indices] = 1
        c_overlay = self.mask[self.indices][...,None]*[1.,0,0]*self.mask_alpha
        array[self.indices] = (c_overlay + self.img[self.indices]*(1-self.mask_alpha))

        self.displayed.set_data(array)
        self.fig.canvas.draw_idle()
        
    def _ipython_display_(self):
        display(self.fig.canvas)

In [43]:
import cv2

# Load an image
image_path = '/data/share/innocorn/datasets/patchedDirtySacch/no_sacch/IMG_0005_patch_132.jpg'  # Replace with your image path
image = cv2.imread(image_path)

In [145]:
# Create an instance of the image lasso selector
selector = image_lasso_selector(image)

# Display the interactive plot
selector._ipython_display_()

NameError: name 'image_lasso_selector' is not defined

In [116]:
from collections import defaultdict 
data_dict = defaultdict(list)

In [102]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector

class rectangular_selector:
  """
  This class allows users to select a rectangular region of an image.

  Args:
      img: A numpy array representing the image with shape (X, Y, 3).
      figsize: A tuple of (width, height) for the figure window (default=(10, 10)).

  """
  def __init__(self, img, mask_alpha=.75, figsize=(10, 10)):
    self.img = img
    self.mask_alpha = mask_alpha
    self.fig = plt.figure(figsize=figsize)
    self.ax = self.fig.gca()
    self.displayed = self.ax.imshow(img)
    # Initialize empty rectangle properties
    self.rect_props = {'facecolor': 'blue', 'alpha': 0.3, 'linestyle': 'dashed'}
    self.rect = None  # This will store the rectangle object

    # Create rectangle selector and connect it to the onselect function
    self.selector = RectangleSelector(self.ax, self.onselect, interactive=True, props=self.rect_props)
    
  def onselect(self, eclick, erelease):
    """
    This function is called whenever the user finishes selecting a rectangle.

    Args:
        eclick: A Matplotlib event object containing the starting click coordinates.
        erelease: A Matplotlib event object containing the ending release coordinates.
    """
    # Extract coordinates from events
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata

    # Ensure proper rectangle (top-left to bottom-right)
    if x1 > x2:
      x1, x2 = x2, x1
    if y1 > y2:
      y1, y2 = y2, y1
    coords = (x1, y1, x2, y2)
    # Draw the rectangle on the image
    self.draw_rectangle(x1, y1, x2, y2)
    # add the rectangle coords to the dictionary
    data_dict[image_paths[ix]].append(coords)

  def draw_rectangle(self, x1, y1, x2, y2):
    """
    This function draws a rectangle on the image based on the provided coordinates.

    Args:
        x1: X coordinate of the top-left corner.
        y1: Y coordinate of the top-left corner.
        x2: X coordinate of the bottom-right corner.
        y2: Y coordinate of the bottom-right corner.
    """
    # Clear any existing rectangle
    if self.rect is not None:
      self.rect.remove()

    # Create a rectangle patch with given coordinates and properties
    width = x2 - x1
    height = y2 - y1
    self.rect = plt.Rectangle(xy=(x1, y1), width=width, height=height, **self.rect_props)

    # Add the rectangle to the axis and redraw the figure
    self.ax.add_patch(self.rect)
    self.fig.canvas.draw_idle()

  def _ipython_display_(self):
        display(self.fig.canvas)

In [14]:
import cv2
image_paths = [
    '/data/share/innocorn/datasets/patchedDirtySacch/no_sacch/IMG_0005_patch_132.jpg',
    '/data/share/innocorn/datasets/patchedDirtySacch/no_sacch/IMG_0005_patch_131.jpg',
    '/data/share/innocorn/datasets/patchedDirtySacch/sacch/IMG_0011_patch_22.jpg',
    '/data/share/innocorn/datasets/patchedDirtySacch/sacch/IMG_0011_patch_5.jpg',
]
images = [cv2.imread(path) for path in image_paths]

In [18]:
%matplotlib widget
import ipywidgets
from IPython.display import display, clear_output
button = ipywidgets.Button(description="Click Me!")
output = ipywidgets.Output()

display(button, output)

def on_button_clicked(b):
    global ix
    global button
    with output:
        ix += 1
        button.description = f"Image {ix}"
        # do something here (e.g. process_some_data(ix))
ix = 0
button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

In [23]:
selector = rectangular_selector(images[ix])

IndexError: list index out of range

In [24]:
data_dict

defaultdict(list,
            {'/data/share/innocorn/datasets/patchedDirtySacch/no_sacch/IMG_0005_patch_132.jpg': [(42.28302556818181,
               26.72908380681818,
               73.41029829545455,
               56.401811079545496)],
             '/data/share/innocorn/datasets/patchedDirtySacch/no_sacch/IMG_0005_patch_131.jpg': [(110.06484375000001,
               33.88545809659095,
               185.11938920454543,
               87.70363991477276)],
             '/data/share/innocorn/datasets/patchedDirtySacch/sacch/IMG_0011_patch_22.jpg': [(136.5375710227273,
               156.41636186079546,
               223.22848011363638,
               223.0345436789773)],
             '/data/share/innocorn/datasets/patchedDirtySacch/sacch/IMG_0011_patch_5.jpg': [(0.39211647727272947,
               0.5472745028409634,
               98.13757102272726,
               139.89272904829548)]})

In [25]:
import json
data_json = json.dumps(data_dict)

In [26]:
data_json

'{"/data/share/innocorn/datasets/patchedDirtySacch/no_sacch/IMG_0005_patch_132.jpg": [[42.28302556818181, 26.72908380681818, 73.41029829545455, 56.401811079545496]], "/data/share/innocorn/datasets/patchedDirtySacch/no_sacch/IMG_0005_patch_131.jpg": [[110.06484375000001, 33.88545809659095, 185.11938920454543, 87.70363991477276]], "/data/share/innocorn/datasets/patchedDirtySacch/sacch/IMG_0011_patch_22.jpg": [[136.5375710227273, 156.41636186079546, 223.22848011363638, 223.0345436789773]], "/data/share/innocorn/datasets/patchedDirtySacch/sacch/IMG_0011_patch_5.jpg": [[0.39211647727272947, 0.5472745028409634, 98.13757102272726, 139.89272904829548]]}'

In [28]:
with open("test.json", "w") as outfile:
  json.dump(data_json, outfile)